Первый шаг - подготовка окружения и загрузка данных

In [3]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

data = load_boston()
df = pd.read_csv(data['filename'], skiprows=1)

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Теперь не плохо было бы изучить датасет

In [2]:
print(data['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Итого по полям:

- CRIM - уровень преступности на душу населения по городам
- ZN - доля жилой земли, зонированной на участки свыше 25 000 кв. м.фут.
- INDUS - доля акров не-розничного бизнеса в городе
- CHAS - Фиктивная переменная реки Чарльз (1, Если тракт ограничивает реку; 0 если нет)
- NOX - концентрация оксидов азота (частей на 10 миллионов)
- RM - среднее количество комнат в жилом помещении
- AGE - доля занятых владельцами единиц, построенных до 1940 года
- DIS - взвешенные расстояния до пяти бостонских центров занятости
- RAD - индекс доступности к радиальным магистралям
- TAX - налог на недвижимости в \$10,000
- PTRATIO - соотношение учеников и учителей по городам
- B - 1000 (Bk - 0.63)^2 где Bk-доля чернокожих по городам
- LSTAT - \% более низкий статус населения
- MEDV - Медианная стоимость занимаемых владельцами домов в \$1000

Осмотрев поверхностно информацию о данных, я осмелюсь предположить, что рассматриваются именно жилые дома и цены, соответственно, на жильё. У нас есть различные колонки, но все они представляют собой какие-то числовые значения. Попробую разобрать, что может влиять на стоимость жилых помещений и в какой степени.

### CRIM - уровень преступности на душу населения по окрестностям
Однозначно, это важный показатель, влияющий на снижение стоимости жилой недвижимости! Так же, уровень приступности возможно связан с низким уровнем образования и инфрастуктуры в квартале.

### ZN - доля жилой земли, зонированной на участки свыше 25 000 кв. кв.фут.
Чтобы понять, что это за показатель, пришлось взглянуть на карту Бостона. На первый взгляд город выглядит, словно его рисовали по линейке. Мне кажется, что это показатель доли жилых зданий на квартал в 25 тыс. футов$^2$. 

Высокая доля жилой земли говорит о приближении к спальным районам, которые, как правило, располагаются на окраине. Черезмерно высокая доля жилой земли говорит о меньшем количистве необходимой инфраструктуры рядом с жилыми домами.

### INDUS - доля акров не-розничного бизнеса по окрестностям
Показатель не способствует повышению благ для близживущего населения. Скорее всего, это повышает стоимость недвижимости в общем, т.к. крупный бизнес "кучкуется" в центре, либо забирает что-то вроде складов/портов/вокзалов. Двоякий показатель, может как увеличить, так и уменьшить стоимость недвижимости.

### CHAS - Фиктивная переменная реки Чарльз (1, Если тракт ограничивает реку; 0 если нет)
Опять же, чтобы понять, что это за показатель, пришлось поизучать карту Бостона. Если верить границам на [Google Maps](https://www.google.com/maps/place/%D0%91%D0%BE%D1%81%D1%82%D0%BE%D0%BD,+%D0%9C%D0%B0%D1%81%D1%81%D0%B0%D1%87%D1%83%D1%81%D0%B5%D1%82%D1%81,+%D0%A1%D0%A8%D0%90/@42.3145186,-71.1103703,11z/data=!3m1!4b1!4m5!3m4!1s0x89e3652d0d3d311b:0x787cbf240162e8a0!8m2!3d42.3600825!4d-71.0588801), то город в северной своей части разделяется на 3 части рекой. При чём побережье плотно забито различной транспортной инфраструктурой - порт, аэропорт, крупные транспортные развязки.

На первый взгляд (до изучения карты), я хотел исключить данный показатель из датасета. Но увидев карту я заметил там такие постройки, которые способствуют ухудшению уровня жизни вблизи с ним. Порт, аэропорт, ж\д вокзал помимо шума привлекают массу грязи, пыли и не приятных людей. Я уверен, что этот показатель нужно оставить, как сильно влияющий на стоимость жилой недвижимости вблизи с рекой Чарльз. 

### NOX - концентрация оксидов азота (частей на 10 миллионов)
В "интернетах" написано, цитирую: "Воздействие оксидов азота на человека приводит к нарушения функций легких и бронхов . Воздействию оксидов азота в большей степени дети и взрослые, страдающие сердечно — сосудистыми заболеваниями."

Я уверен, что люди об этом знают и принимают во внимание этот фактор. Он значительно ухудшает здоровье, следовательно, должен негативно сказаться на стоимости жилья.

### RM - среднее количество комнат в жилом помещении
Показатель рассказывает нам скорее о размере того или иного помещения. Обычно, количетсво комнат положительно сказывается на стоимости жилья.

### AGE - доля занятых владельцами помещений, построенных до 1940 года
Как мне кажется, показатель сказывается на общем возрасте зданий в округе. Как мне кажется, этот показатель не влияет на результат продсказания целевой метрики, т.к. высокий AGE может повысить стоимость из-за снижения предложения свободного жилья, но понизит, в связи с устаревшими постройками.

Старость постройки может сказаться ухудшенными показателями жилого дома. Я этот столбец исключу.

### DIS - взвешенные расстояния до пяти бостонских центров занятости
Тоже размытый (на первый взгляд) показатель, но спасибо Google Maps (опять же). Простой запрос в гугл показал, что эти центры занятости расположены кучно в центре города.

Большая удалённость от центра города сказывается негативно на стоимости и это важный показатель.

### RAD - индекс доступности к радиальным магистралям
Показатель тоже раскрылся только на картах. Все эти магистрали расположены в северной части города. Как и сам центр, они смещены на север. Данный показатель помимо удобств указывает и на отдалённость от центра. Его тоже можно оставить, т.к. он должен влиять на стоимость жилья

### TAX - налог на недвижимости в \$10,000
А вот этот показатель крайне не понятен мне. Я не понимаю, включается ли это в стоимость на недвижимость или же он наоборот заставляет стоимость снизиться.

Для принятия решения по судьбе данного показателя, я нагуглил [онлайн-калькулятор](https://smartasset.com/taxes/massachusetts-property-tax-calculator#6EX8sGkfCd) налогов в зависимости от цены на недвижимость в штате Массачусетс. Он показывает, что налоги прямо пропорциональны стоимости на недвижимость.

Поэтому данный признак оставляю

### PTRATIO - соотношение учеников и учителей по окрестностям
Казалось бы, этот показатель влияет на перспективы образования и должен повлиять на привлекательность недвижимости в районе с большим количетсвом учителей, но не все люди имеют детей. Вдобавок, многие возят детей учиться зачастую в отдалённые от местности заведения. Этот показатель исключу

### B - 1000 (Bk - 0.63)^2 где Bk-доля чернокожих по окрестностям
Как бы не хотелось сказать: "Да что за дескриминация!", но этот показатель может повлиять на стоимость жилья. Не из-за рассовых предрассудков, а из за статистики, которая показывает, что густо населённое афроамериканцами кварталы имеют более низкий уровень образования и высокий уровень преступности.

### LSTAT - \% более низкий статус населения
Определенно влияющий на результат показатель. Статусное население обычно окружает себя такими же статусными людьми, что зачастую достигается за счёт повышения цен, включая цены на недвижимость.

### MEDV - Медианная стоимость домов в \$1000
Наше целевое значение. Показывает среднее значение стоимости домой

Исходя из анализа колонок, я принимаю решение избавиться от "AGE", "PTRATIO"

In [14]:
df = df.drop(['AGE', 'PTRATIO'], axis=1)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,4.0900,1,296,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,4.9671,2,242,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,4.9671,2,242,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,6.0622,3,222,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,6.0622,3,222,396.90,5.33,36.2


Наши данные избавлены от лишних столбцов. Теперь необходимо провести поверхностный анализ данных на наличие пустых значений

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 12 columns):
CRIM     506 non-null float64
ZN       506 non-null float64
INDUS    506 non-null float64
CHAS     506 non-null int64
NOX      506 non-null float64
RM       506 non-null float64
DIS      506 non-null float64
RAD      506 non-null int64
TAX      506 non-null int64
B        506 non-null float64
LSTAT    506 non-null float64
MEDV     506 non-null float64
dtypes: float64(9), int64(3)
memory usage: 47.6 KB


Данные выглядят нормальными и без повреждений. Так же, все типы приведены к какому-то строковому представлению, что позволяет нам начать анализ на наличие выбросов.

In [16]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,3.795043,9.549407,408.237154,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,2.105710,8.707259,168.537116,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,1.129600,1.000000,187.000000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,2.100175,4.000000,279.000000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,3.207450,5.000000,330.000000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,5.188425,24.000000,666.000000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,12.126500,24.000000,711.000000,396.900000,37.970000,50.000000


### CRIM
Выбросы определенно есть, т.к. максимальное значение намного выше среднего или сигмы > 75%

**Требуется проверка**


### ZN
Так же, очень похоже, что есть выброс, т.к. значение сильно отклоняется от среднего и каждой из сигм. Но если брать во внимание специфичность данного аттрибута, то значение вполне допустимое. Есть нулевые значения

**Требуется проверка**


### INDUS
Отклонение максимального значения от среднего или сигмы >75% значительное, но выглядит нормальным. Проверю на всякий случай этот аттрибут

**Требуется проверка**


### CHAS
Судя по специфичности аттрибута, все значения в норме. Нулевые значения присутствуют, но они допустимы

**Проверка не требуется**


### NOX
Все среднестатистические показатели в норме

**Проверка не требуется**


### RM
Среднестатистические показатели выглядят нормальными

**Проверка не требуется**


### DIS
Максимальное значение намного выше, чем среднестатистическое и сигма >75%

**Требуется проверка**


### RAD
Все показатели выглядят нормальными

**Проверка не требуется**


### TAX
Все показатели выглядят нормальными

**Проверка не требуется**


### B
Минимальное значение не выглядит естесственно

**Требуется проверка**


### LSTAT
Максимальное значение выглядит весьма выше, чем сигма >75%

**Требуется проверка**


### MEDV
Все показатели выглядят нормальными

**Проверка не требуется**

### Проверим CRIM
Требуется проверка максимальных значений

In [28]:
print(df[df['CRIM'] > 10].shape) # 10 - выше стандартного отклонения
df[df['CRIM'] > 10].head()

(54, 12)


,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
367,13.5222,0.0,18.1,0,0.631,3.863,1.5106,24,666,131.42,13.33,23.1
373,11.1081,0.0,18.1,0,0.668,4.906,1.1742,24,666,396.90,34.77,13.8
374,18.4982,0.0,18.1,0,0.668,4.138,1.1370,24,666,396.90,37.97,13.8
375,19.6091,0.0,18.1,0,0.671,7.313,1.3163,24,666,396.90,13.44,15.0
376,15.2880,0.0,18.1,0,0.671,6.649,1.3449,24,666,363.02,23.24,13.9


По моему скромному наблюдению, есть очень странная закономерность. Во всех 54 найденных записях, где уровень приступности выше 10 процентов, налог равен 666 О_о

Совпадение? Не думаю!

Или, это было неожидано и не до конца ясно, что с этим делать... Попробую найти записи сболее высоким уровнем приступности

In [31]:
df[df['CRIM'] > 50].reset_index()

,index,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
0,380,88.9762,0.0,18.1,0,0.671,6.968,1.4165,24,666,396.90,17.21,10.4
1,405,67.9208,0.0,18.1,0,0.693,5.683,1.4254,24,666,384.97,22.98,5.0
2,410,51.1358,0.0,18.1,0,0.597,5.757,1.4130,24,666,2.60,10.11,15.0
3,418,73.5341,0.0,18.1,0,0.679,5.957,1.8026,24,666,16.45,20.62,8.8


Итого, всего 4 записи. Я думаю, что можно исключать записи с уровнем приступности выше 50%, т.к. это может сильно повлиять на нашу модель

In [35]:
df = df[df['CRIM'] < 50]

df.shape

(502, 12)

Отлично, записей стало меньше и мы исключили черезмерно высокие по криминалитету записи

### Проверим ZN

Требуется проверка минимального и максимального значений

In [52]:
df[df['ZN'] == 0].shape # сигма >75%

(368, 12)

In [53]:
df[df['ZN'] == 0].head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
1,0.02731,0.0,7.07,0,0.469,6.421,4.9671,2,242,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,4.9671,2,242,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,6.0622,3,222,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,6.0622,3,222,396.90,5.33,36.2
5,0.02985,0.0,2.18,0,0.458,6.430,6.0622,3,222,394.12,5.21,28.7


Удивительный результат. Оказывается в нашем датасете больше половины записей не имеют жилой площади... Это очень странно, т.к. и показатель INDUS, отвечающий за процентное соотношение крупного бизнеса на территории весьма низкое. С трудом могу себе представить, что же это такое означает

Возможно первоначальная гипотеза о том, что мы оцениваем стоимость только жилой недвижимости не верная. Да, закреплю такой вывод. Удалять нулевые значения я не буду, т.к. это сломает выборку

Теперь проверим значения выше 80%

In [45]:
df[df['ZN'] > 80].shape

(14, 12)

Есть 14 таких записей, изучим их

In [47]:
df[df['ZN'] > 80]

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
55,0.01311,90.0,1.22,0,0.4030,7.249,8.6966,5,226,395.93,4.81,35.4
56,0.02055,85.0,0.74,0,0.4100,6.383,9.1876,2,313,396.90,5.77,24.7
57,0.01432,100.0,1.32,0,0.4110,6.816,8.3248,5,256,392.90,3.95,31.6
199,0.03150,95.0,1.47,0,0.4030,6.975,7.6534,3,402,396.90,4.56,34.9
200,0.01778,95.0,1.47,0,0.4030,7.135,7.6534,3,402,384.30,4.45,32.9
201,0.03445,82.5,2.03,0,0.4150,6.162,6.2700,2,348,393.77,7.43,24.1
202,0.02177,82.5,2.03,0,0.4150,7.610,6.2700,2,348,395.38,3.11,42.3
203,0.03510,95.0,2.68,0,0.4161,7.853,5.1180,4,224,392.78,3.81,48.5
204,0.02009,95.0,2.68,0,0.4161,8.034,5.1180,4,224,390.55,2.88,50.0
256,0.01538,90.0,3.75,0,0.3940,7.454,6.3361,3,244,386.34,3.11,44.0


Показатели выглядят нормальными, оставим все значения

### Проверим INDUS
Требуется проверка максимального значения

In [54]:
df[df['INDUS'] > 25] # 25 - близкое к максимуму значение

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
120,0.06899,0.0,25.65,0,0.581,5.870,2.2577,2,188,389.15,14.37,22.0
121,0.07165,0.0,25.65,0,0.581,6.004,2.1974,2,188,377.67,14.27,20.3
122,0.09299,0.0,25.65,0,0.581,5.961,2.0869,2,188,378.09,17.93,20.5
123,0.15038,0.0,25.65,0,0.581,5.856,1.9444,2,188,370.31,25.41,17.3
124,0.09849,0.0,25.65,0,0.581,5.879,2.0063,2,188,379.38,17.58,18.8
125,0.16902,0.0,25.65,0,0.581,5.986,1.9929,2,188,385.02,14.81,21.4
126,0.38735,0.0,25.65,0,0.581,5.613,1.7572,2,188,359.29,27.26,15.7
488,0.15086,0.0,27.74,0,0.609,5.454,1.8209,4,711,395.09,18.06,15.2
489,0.18337,0.0,27.74,0,0.609,5.414,1.7554,4,711,344.05,23.97,7.0
490,0.20746,0.0,27.74,0,0.609,5.093,1.8226,4,711,318.43,29.68,8.1


Подобные значения - норма, значит можно оставить записи не тронутыми

### Проверим DIS
Проверим максимальное значение

In [57]:
df[df['DIS'] > 10] # 10 - близкое к максимуму

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
351,0.07950,60.0,1.69,0,0.411,6.579,10.7103,4,411,370.78,5.49,24.1
352,0.07244,60.0,1.69,0,0.411,5.884,10.7103,4,411,392.33,7.79,18.6
353,0.01709,90.0,2.02,0,0.410,6.728,12.1265,5,187,384.46,4.50,30.1
354,0.04301,80.0,1.91,0,0.413,5.663,10.5857,4,334,382.80,8.05,18.2
355,0.10659,80.0,1.91,0,0.413,5.936,10.5857,4,334,376.04,5.57,20.6


Значение выглядит естественным, изменения не требуются

### Проверим B
Минимальное значение этого показателя выглядит подозрительным

In [62]:
df[df['B'] < 10] # 10 - значение сильно отличающееся от среднего и близкое к нулю

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
423,7.05042,0.0,18.1,0,0.614,6.103,2.0218,24,666,2.52,23.29,13.4
424,8.79212,0.0,18.1,0,0.584,5.565,2.0635,24,666,3.65,17.16,11.7
425,15.86030,0.0,18.1,0,0.679,5.896,1.9096,24,666,7.68,24.39,8.3
437,15.17720,0.0,18.1,0,0.740,6.152,1.9142,24,666,9.32,26.45,8.7
450,6.71772,0.0,18.1,0,0.713,6.749,2.3236,24,666,0.32,17.44,13.4
454,9.51363,0.0,18.1,0,0.713,6.728,2.4961,24,666,6.68,18.71,14.9
457,8.20058,0.0,18.1,0,0.713,5.936,2.7792,24,666,3.50,16.94,13.5


Показатели выглядят подозрительно, но вполне возможно. Оставлю без изменений

### Проверим LSTAT
Проверим максимальное значение в выборке и его отклонение от нормы

In [66]:
df[df['LSTAT'] > 30] # 30 - близкое к максимальному

,CRIM,ZN,INDUS,CHAS,NOX,RM,DIS,RAD,TAX,B,LSTAT,MEDV
48,0.25387,0.0,6.91,0,0.448,5.399,5.8700,3,233,396.90,30.81,14.4
141,1.62864,0.0,21.89,0,0.624,5.019,1.4394,4,437,396.90,34.41,14.4
373,11.10810,0.0,18.10,0,0.668,4.906,1.1742,24,666,396.90,34.77,13.8
374,18.49820,0.0,18.10,0,0.668,4.138,1.1370,24,666,396.90,37.97,13.8
384,20.08490,0.0,18.10,0,0.700,4.368,1.4395,24,666,285.83,30.63,8.8
385,16.81180,0.0,18.10,0,0.700,5.277,1.4261,24,666,396.90,30.81,7.2
387,22.59710,0.0,18.10,0,0.700,5.000,1.5184,24,666,396.90,31.99,7.4
388,14.33370,0.0,18.10,0,0.700,4.880,1.5895,24,666,372.92,30.62,10.2
398,38.35180,0.0,18.10,0,0.693,5.453,1.4896,24,666,396.90,30.59,5.0
412,18.81100,0.0,18.10,0,0.597,4.628,1.5539,24,666,28.79,34.37,17.9


Распределение выглядит нормальным, измнения не требуются

**Итого:**

После проверки на выбросы и качество данных, мы избавились от небольшого количества записей. Это говорит о том, что данные не плохого качества и что мы могли угадать с лишними столбцами

### Обучение модели

Разделим данные на тренировочные и тестовые

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X = df.drop(['MEDV'], axis=1)
y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Построим модель линейной регрессии и обучим её на тестовых данных

In [86]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Проверим как обучилась наша модель

In [87]:
pred = model.predict(X_test)

print('mean_absolute_error:', metrics.mean_absolute_error(y_test, pred))

mean_absolute_error: 3.525725345078412


### Итого

Учитывая, что min(MEDV) == 5, а max(MEDV) == 50.000000, то у меня получилась не большая средняя абсолютная ошибка, и модель обучилась довольно качественно